In [1]:
from bs4 import BeautifulSoup as bs
import requests

response = requests.get('https://mars.nasa.gov/news/').text

soup = bs(response, 'lxml')


In [2]:
#NASA Mars News ALL
import pandas as pd

news =[]

for result in soup.find_all('div', class_ ="slide"):
    body = result.find('div', class_ = 'rollover_description_inner').text
    
    title = result.find('div', class_ = 'content_title').text
        
    news.append({'title':title.replace("\n", ""), 'body':body.replace("\n", "")})
    

    
df_news = pd.DataFrame(news)

df_news = df_news[['title','body']]

df_news

    

,title,body
0,NASA's Mars 2020 Rover Closer to Getting Its Name,155 students from across the U.S. have been ch...
1,NASA Invites Students to Name Mars 2020 Rover,"Through Nov. 1, K-12 students in the U.S. are ..."
2,NASA's Curiosity Mars Rover Finds a Clay Cache,"The rover recently drilled two samples, and bo..."
3,Why This Martian Full Moon Looks Like Candy,"For the first time, NASA's Mars Odyssey orbite..."
4,NASA Garners 7 Webby Award Nominations,Nominees include four JPL projects: the solar ...
5,NASA's Opportunity Rover Mission on Mars Comes...,NASA's Opportunity Mars rover mission is compl...


In [3]:
#NASA Mars News MOST Current
import pandas as pd

news_title = soup.find('div', class_ = 'content_title').text

news_p = soup.find('div', class_ = 'rollover_description_inner').text
    
print(news_title,news_p)





NASA's Mars 2020 Rover Closer to Getting Its Name

 
155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.



In [4]:
#JPL Mars Space Image url
from splinter import Browser

executable_path = {'executable_path':'chromedriver.exe'}

browser = Browser('chrome', **executable_path)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&catagory=Mars'

browser.visit(url)

browser.find_by_css('footer').click()

browser.find_by_xpath('//*[@id="fancybox-lock"]/div/div[2]/div/div[1]/a[2]').click()

browser.find_by_css('figure').click()

featured_image_url = browser.url

print(featured_image_url)

browser.quit()

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg


In [6]:
#Mars Weather from twitter
executable_path = {'executable_path':'chromedriver.exe'}

browser = Browser('chrome', **executable_path)

url = 'https://twitter.com/marswxreport?lang=en'

browser.visit(url)

response = requests.get('https://twitter.com/marswxreport?lang=en').text

soup = bs(response, 'lxml')

num = 1

for result in soup.find_all('p', class_ ="tweet-text"):
    result = str(result)
    
    if "InSight" in result:
        start = 'InSight '
        end = '<a'   
        mars_weather = (result[result.find(start)+len(start):result.rfind(end)])
        print(mars_weather)
        break
    else:
        num = num + 1
        if num > 20:
            break
            
browser.quit()

sol 411 (2020-01-22) low -96.4ºC (-141.4ºF) high -16.7ºC (1.9ºF)
winds from the S at 5.6 m/s (12.6 mph) gusting to 22.7 m/s (50.8 mph)
pressure at 6.30 hPa


In [7]:
# Visit the Mars facts webpage and scrape table data into Pandas
from splinter import Browser
import pandas as pd 
import time

executable_path = {'executable_path':'chromedriver.exe'}

browser = Browser('chrome', **executable_path)

url = 'https://space-facts.com/mars/'

browser.visit(url)

# place data into a dataframe output to an HTML string
grab=pd.read_html(url)
df_mars_data=pd.DataFrame(grab[0])
df_mars_data.columns=['Mars','Data']
df_mars_table=df_mars_data.set_index("Mars")
marsdata_html = df_mars_table.to_html(classes='marsdata')
marsdata_html=marsdata_html.replace('\n', ' ')

print(marsdata_html)

browser.quit()


<table border="1" class="dataframe marsdata">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Data</th>     </tr>     <tr>       <th>Mars</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.39 × 10^23 kg (0.11 Earths)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.38 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-87 to -5 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>


In [8]:
#Mars Hemisphers
from splinter import Browser
import pandas as pd 
import time

executable_path = {'executable_path':'chromedriver.exe'}

browser = Browser('chrome', **executable_path)

url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)

mars_hemis=[]


for i in range (4):
    time.sleep(5)
    images = browser.find_by_tag('h3')
    images[i].click()
    new_url = browser.url
    response = requests.get(new_url).text
    soup = bs(response, 'lxml')
    partial = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial
    dictionary={"title":img_title,"img_url":img_url}
    mars_hemis.append(dictionary)
    browser.back()
   


In [9]:
print(mars_hemis)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
